# Text-to-SQL Query Generation

This notebook presents a Python-based tool designed to simplify querying credit card fraud data by converting natural language queries into SQL using the GPT API. With this tool, users can easily interact with the dataset without needing SQL expertise.

Develop a Python-based application within a Jupyter Notebook that uses the GPT API to convert natural language queries into SQL queries. These SQL queries should be executed on a database containing credit card fraud data, and the results should be displayed as a DataFrame.

Dataset: You will be using the Credit Card Fraud dataset available at: Credit Card Fraud Data. link: https://www.kaggle.com/datasets/neharoychoudhury/credit-card-fraud-data

#### Key Features:

    - Database Setup: Load the credit card fraud data into an SQLite database for structured querying.
    - Text-to-SQL: Leverage GPT to automatically convert natural language questions into SQL queries.
    - Query Execution & Results: Execute SQL queries on the database and display results in a pandas DataFrame.
    - SQL Query Display: The generated SQL query is shown alongside the query results for better understanding.

This tool enhances fraud data analysis, making it accessible and efficient, even for users unfamiliar with SQL.

# Import necessary libraries

In [ ]:
!pip install openai

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from openai import OpenAI

# Read the data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submitted CPSC final exam/fraud_data.csv')

df.head()


,trans_date_trans_time,merchant,category,amt,city,state,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud
0,04-01-2019 00:58,"""Stokes, Christiansen and Sipes""",grocery_net,14.37,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,a3806e984cec6ac0096d8184c64ad3a1,65.654142,-164.722603,1
1,04-01-2019 15:06,Predovic Inc,shopping_net,966.11,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,a59185fe1b9ccf21323f581d7477573f,65.468863,-165.473127,1
2,04-01-2019 22:37,Wisozk and Sons,misc_pos,49.61,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,86ba3a888b42cd3925881fa34177b4e0,65.347667,-165.914542,1
3,04-01-2019 23:06,Murray-Smitham,grocery_pos,295.26,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,3a068fe1d856f0ecedbed33e4b5f4496,64.445035,-166.080207,1
4,04-01-2019 23:59,Friesen Lt,health_fitness,18.17,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,891cdd1191028759dc20dc224347a0ff,65.447094,-165.446843,1


Create a new column "year" to extract the year only.

In [ ]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'], format='%d-%m-%Y %H:%M')

df['year'] = df['trans_date_trans_time'].dt.year

print(df.head())

  trans_date_trans_time                          merchant        category  \
0   2019-01-04 00:58:00  "Stokes, Christiansen and Sipes"     grocery_net   
1   2019-01-04 15:06:00                      Predovic Inc    shopping_net   
2   2019-01-04 22:37:00                   Wisozk and Sons        misc_pos   
3   2019-01-04 23:06:00                    Murray-Smitham     grocery_pos   
4   2019-01-04 23:59:00                        Friesen Lt  health_fitness   

      amt   city state      lat      long  city_pop  \
0   14.37  Wales    AK  64.7556 -165.6723       145   
1  966.11  Wales    AK  64.7556 -165.6723       145   
2   49.61  Wales    AK  64.7556 -165.6723       145   
3  295.26  Wales    AK  64.7556 -165.6723       145   
4   18.17  Wales    AK  64.7556 -165.6723       145   

                          job         dob                         trans_num  \
0  "Administrator, education"  09-11-1939  a3806e984cec6ac0096d8184c64ad3a1   
1  "Administrator, education"  09-11-1939  a5918

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14446 entries, 0 to 14445
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   trans_date_trans_time  14446 non-null  datetime64[ns]
 1   merchant               14446 non-null  object        
 2   category               14446 non-null  object        
 3   amt                    14446 non-null  float64       
 4   city                   14446 non-null  object        
 5   state                  14446 non-null  object        
 6   lat                    14446 non-null  float64       
 7   long                   14446 non-null  float64       
 8   city_pop               14446 non-null  int64         
 9   job                    14446 non-null  object        
 10  dob                    14446 non-null  object        
 11  trans_num              14446 non-null  object        
 12  merch_lat              14446 non-null  float64       
 13  m

# Load Data into SQLite Database

In [ ]:
conn = sqlite3.connect('fraud_database.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS FRAUD_TABLE (
        trans_date_trans_time TEXT,
        merchant TEXT,
        category TEXT,
        amt REAL,
        city TEXT,
        state TEXT,
        lat REAL,
        long REAL,
        city_pop INTEGER,
        job TEXT,
        dob TEXT,
        trans_num TEXT,
        merch_lat REAL,
        merch_long REAL,
        is_fraud INTEGER,
        year INTEGER
    )
''')

In [ ]:
df.to_sql('FRAUD_TABLE', conn, if_exists='replace', index=False, dtype={
    'trans_date_trans_time': 'TEXT',
    'merchant': 'TEXT',
    'category': 'TEXT',
    'amt': 'REAL',
    'city': 'TEXT',
    'state': 'TEXT',
    'lat': 'REAL',
    'long': 'REAL',
    'city_pop': 'INTEGER',
    'job': 'TEXT',
    'dob': 'TEXT',
    'trans_num': 'TEXT',
    'merch_lat': 'REAL',
    'merch_long': 'REAL',
    'is_fraud': 'INTEGER',
    'year': 'INTEGER'
})

# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:

# Open a new connection to the SQLite database
conn = sqlite3.connect('fraud_database.db')
cursor = conn.cursor()

# Check the schema
cursor.execute('PRAGMA table_info(FRAUD_TABLE)')
schema_info = cursor.fetchall()

# Print schema information
for column in schema_info:
    print(column)

# Close the connection
conn.close()


(0, 'trans_date_trans_time', 'TEXT', 0, None, 0)
(1, 'merchant', 'TEXT', 0, None, 0)
(2, 'category', 'TEXT', 0, None, 0)
(3, 'amt', 'REAL', 0, None, 0)
(4, 'city', 'TEXT', 0, None, 0)
(5, 'state', 'TEXT', 0, None, 0)
(6, 'lat', 'REAL', 0, None, 0)
(7, 'long', 'REAL', 0, None, 0)
(8, 'city_pop', 'INTEGER', 0, None, 0)
(9, 'job', 'TEXT', 0, None, 0)
(10, 'dob', 'TEXT', 0, None, 0)
(11, 'trans_num', 'TEXT', 0, None, 0)
(12, 'merch_lat', 'REAL', 0, None, 0)
(13, 'merch_long', 'REAL', 0, None, 0)
(14, 'is_fraud', 'INTEGER', 0, None, 0)
(15, 'year', 'INTEGER', 0, None, 0)


Check the table in the database

In [ ]:

def list_tables(db_path):
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Execute a query to get all table names
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

        # Fetch all results
        tables = cursor.fetchall()

        # Close the connection
        conn.close()

        # Return the list of table names
        return tables
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Path to your SQLite database
db_path = 'fraud_database.db'
tables = list_tables(db_path)

if tables is not None:
    print("Tables in the database:")
    for table in tables:
        print(table[0])
else:
    print("Could not retrieve tables.")


Tables in the database:
FRAUD_TABLE


# Set up API key

In [3]:
api_key = '[your key here]'

#### Text-to-SQL Functionality

Text-to-SQL Functionality:

Uses the GPT API to convert a natural language query into an SQL query.
        
        
client.chat.completions.create sends the prompt to the GPT model and retrieves the generated SQL query.

In [ ]:
def text_to_sql(natural_language_query):
    prompt = [
        {
            "role": "system",
            "content": "You are provided a table in the database is named FRAUD_TABLE. Columns include trans_date_trans_time (TEXT), merchant (TEXT), category (TEXT), amt (REAL), city (TEXT), state (TEXT), lat (REAL), long (REAL), city_pop (INTEGER), job (TEXT), dob (TEXT), trans_num (TEXT), merch_lat (REAL), merch_long (REAL), and is_fraud (INTEGER) and your task is to convert natural language questions into SQL queries. "
        },
        {
            "role": "user",
            "content": f"Convert the following natural language question into an SQL query: {natural_language_query}"
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=prompt
        )
    sql_query = response.choices[0].message.content.strip()
    sql_query = clean_sql_query(sql_query)
    return sql_query


In [ ]:
def clean_sql_query(sql_query):
    """
    Clean the SQL query by removing markdown formatting and extra characters.

    Args:
    sql_query (str): The SQL query string to be cleaned.

    Returns:
    str: The cleaned SQL query.
    """
    # Remove markdown code block formatting (if present)
    cleaned_query = sql_query.replace('```sql', '').replace('```', '').strip()

    # Additional cleaning steps can be added here if necessary
    # e.g., handling newlines or unnecessary spaces

    return cleaned_query

Now create a function to:

- Connects to the SQLite database.
- Executes the SQL query using pd.read_sql_query to retrieve data into a DataFrame.
- Prints the generated SQL query and the results.

In [ ]:
# Execute
def execute_sql_and_display(sql_query):
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect('fraud_database.db')
        cursor = conn.cursor()

        # Print the generated SQL query
        print(f"Generated SQL Query: {sql_query}")

        # Execute the SQL query
        df = pd.read_sql_query(sql_query, conn)

        # Close the connection
        conn.close()

        return df
    except Exception as e:
        print(f"An error occurred: {e}")
        return None



In [ ]:
# Testing

natural_language_query = "Filter the fraud records in the year 2019"
sql_query = text_to_sql(natural_language_query)
df_result = execute_sql_and_display(sql_query)

if df_result is not None:
    print(df_result)


Generated SQL Query: SELECT *
FROM FRAUD_TABLE
WHERE is_fraud = 1 AND strftime('%Y', trans_date_trans_time) = '2019';
     trans_date_trans_time                          merchant        category  \
0      2019-01-04 00:58:00  "Stokes, Christiansen and Sipes"     grocery_net   
1      2019-01-04 15:06:00                      Predovic Inc    shopping_net   
2      2019-01-04 22:37:00                   Wisozk and Sons        misc_pos   
3      2019-01-04 23:06:00                    Murray-Smitham     grocery_pos   
4      2019-01-04 23:59:00                        Friesen Lt  health_fitness   
...                    ...                               ...             ...   
1036   2019-01-18 14:38:00                    Shanahan-Lehne   personal_care   
1037   2019-01-18 22:06:00                         Beier LLC   entertainment   
1038   2019-01-18 22:12:00               Bartoletti and Sons   personal_care   
1039   2019-01-18 23:16:00                 Ankunding-Carroll          travel   
10

   
   # Explain how the GPT API is used for text-to-SQL conversion.

    Input: A natural language question is provided as input.

    Prompt Construction: A prompt is constructed, instructing the GPT model to convert the natural language question into an SQL query.

    Model Response: The GPT model generates a SQL query based on the prompt.
    
    Execution: The generated SQL query is executed against the database to retrieve or manipulate data.

